In [ ]:
import random
import math

In [ ]:
def _es_primo(n, k):
    """
    función que verifica si un número es primo o no, en k iteraciones donde mientras k
    es mayor, la probabilidad aumenta
    """

    if n == 1:
        return False

    elif n == 2:
        return True

    elif n % 2 == 0:
        return False

    else:

        #la parte probabilista con iteraciones
        neg = 0
        for i in range(1, k + 1):

            a = random.randint(2, n - 1)
            if math.gcd(a, n) > 1:
                return False
            else:
                b = pow(a, (n - 1) // 2, n)
                if b == n - 1:
                    neg += 1
                elif b != 1:
                    return False
        if neg > 0:
            return True
        else:
            return False

In [ ]:
def _gen_primes(bit_len):

    """
    genera randoms
    bit_len es el largo del primo
    """
    # daremos un ejemplo para 4 bits
    # 15 -> 2**4  - 1
    # 1111 son 4 bits
    # 1000 - 1111 : 8 a 15

    lo = 2 ** (bit_len) # Como dijimos, todos los numeros con 4 bits van desde 2**3
    hi = 2 ** (bit_len + 1) - 1  #hasta 2**4 - 1

    r1 = random.randint(lo, hi)
    r2 = random.randint(lo, hi)

    while not _es_primo(r1, 100): # generamos primos con prob hasta obtenerlos
        r1 = random.randint(lo, hi)

    while r1 == r2 or not _es_primo(r2, 100):
        r2 = random.randint(lo, hi)

    return r1, r2

In [ ]:
def _primo_relativo(n):
    lo = 2
    hi = n - 1
    pr = random.randint(lo, hi)
    while math.gcd(n, pr) != 1:
        pr = random.randint(lo, hi)
    return pr

In [ ]:
class RSAReceiver :

    def __init__ ( self , bit_len : int ) -> None :
        """
        Arguments :
        bit_len : A lower bound for the number of bits of N ,
        the second argument of the public and secret key .
        """
        self.bit_len = bit_len
        # self.S_A = 0
        # self.P_A = 0
        # 1. Generamos dos prime numbers P y Q, y  definimos N := P*Q.
        P,Q = _gen_primes(self.bit_len)
        self.N = P * Q
        # # 2. Definimos phi(N) := (P-1)*(Q-1)
        phi = (P - 1) * (Q - 1)
        # 3. Generamos d tal que GCD(d,phi(N)) == 1
        self.d = _primo_relativo(phi)
        # 4. Generamos e tal que e*d===1 mod phi(N)
        self.e = pow(self.d, -1, phi)
        # Guardamos llaves publicas y privada
        self.S_A = (self.d, self.N)
        self.P_A = (self.e, self.N)

    def get_public_key ( self ) -> bytearray :
        """
        Returns :
        public_key : Public key expressed as a Python ’ bytearray ’ using the
        PEM format . This means the public key is divided in :
        ( 1 ) The number of bytes of e ( 4 bytes )
        ( 2 ) the number e ( as many bytes as indicated in ( 1 ) )
        ( 3 ) The number of bytes of N ( 4 bytes )
        ( 4 ) the number N ( as many bytes as indicated in ( 3 ) )
        """

        P_A_bytes = bytearray()
        e_nbytes = math.ceil(self.e.bit_length() / 8)
        N_nbytes = math.ceil(self.N.bit_length() / 8)
        P_A_bytes += (
            e_nbytes.to_bytes(4, "big")
            + self.e.to_bytes(e_nbytes, "big")
            + N_nbytes.to_bytes(4, "big")
            + self.N.to_bytes(N_nbytes, "big")
        )
        
        return P_A_bytes
            
    def decrypt ( self , ciphertext : bytearray ) -> str :
        """
        Arguments :
        ciphertext : The ciphertext to decrypt
        Returns :
        message : The original message
        """

        N_int = self.N
        e_int = self.e
        d_int = self.d

        N_nbits = N_int.bit_length()  

        n = 0 

        if (N_nbits//8) * 8 < N_nbits:
            n = N_nbits//8

        else:
            n = (N_nbits//8) - 1 

        n+=1 # sumamos 1 ya que encriptamos los bloques con 1 byte más

        decripted = bytearray()

        # se encripta como establece el protocolo
        for i in range(math.ceil(len(ciphertext)/n)):

            int_cipher = int.from_bytes(ciphertext[n*i:n*(i+1)],'big')
            p= pow(int_cipher, d_int, N_int)             

            ba = p.to_bytes(n+1,'big')

            decripted += ba


        # removemos los elementos nulos
        to_ret = decripted.replace(b'\x00', b'').decode('utf-8')
       
        return to_ret


In [ ]:
class RSASender :
    def __init__ ( self , public_key : bytearray ) -> None :
        """
        Arguments :
        public_key : The public key that will be used to encrypt messages
        """
        self.public_key = public_key

    def encrypt ( self , message : str ) -> bytearray :
        """
        Arguments :
        message : The plaintext message to encrypt
        Returns :
        ciphertext : The encrypted message
        """
        # Para encriptar un mensaje lo separaremos en bloques de n bytes, donde n es el mayor
        # múltiplo de 8 tal que 8 · n es menor que el número de bits de N . Por ejemplo, si para
        # representar N se necesitan 2056 bits, entonces n será 256, dado que 256 · 8 = 2048 y
        # 2048 es el mayor múltiplo de 8 estrictamente menor que 2056.

        
        # tomo el msge y lo transformo a bytearray

        enc_message = bytearray(message, 'utf-8')

        len_enc_message = len(enc_message)
        
        nbytes_e = int.from_bytes(self.public_key[:4],'big')

        e=self.public_key[4:4 + nbytes_e]

        nbytes_N = int.from_bytes(self.public_key[4 + nbytes_e: 8 + nbytes_e],'big')

    
        N = self.public_key[-nbytes_N:]


  

        #PRIMERO: separar el msge en bloques de n bytes
        #como calculo n? -> lo hago desde la public key

        # ¿numero de bits de N? -> len(N)*8

        N_int = int.from_bytes(N,'big')
        e_int = int.from_bytes(e,'big')

        N_nbits =N_int.bit_length() 

        # ahora debo calcular el n

        n = 0 


        if (N_nbits//8) * 8 < N_nbits:
            n = N_nbits//8

        else:
            n = (N_nbits//8) - 1 

        # tengo que sacar los chunks
        cipher = bytearray()

        for i in range(math.ceil(len_enc_message/n)):
          
            int_msge = int.from_bytes(enc_message[n*i:n*(i+1)],'big')
            p= pow(int_msge, e_int, N_int) 

            ba = p.to_bytes(n+1,'big')
            
            cipher += ba

        return cipher


# Tests

In [ ]:
if __name__ == "__main__": 
    juan = RSAReceiver(8)
    pk = juan.get_public_key()

    samuel = RSASender(pk)

    m = "I LA FORTINAIT O La BABA G$$$$+o+o+o+óóóúúú"
    encr = samuel.encrypt(m)

    print(juan.decrypt(encr))
